# MGnify to Darwin Core export notes
## What do OTUs represent in the MGnify pipeline?

Get all SSU downloads for analysis <https://www.ebi.ac.uk/metagenomics/analyses/MGYA00463299> from [Marine metagenomes from the bioGEOTRACES project](https://www.ebi.ac.uk/metagenomics/studies/MGYS00005294#overview):

In [12]:
import pandas as pd
from mgnifyextract.analyses import get_analysis
from mgnifyextract.downloads import FastaDownload, MseqDownload, TsvDownload
from mgnifyextract.studies import get_superstudy_studies
from mgnifyextract.util import clean_taxonomy_string

analysis = get_analysis("MGYA00463299")
analysis

<Analysis https://www.ebi.ac.uk/metagenomics/analyses/MGYA00463299 >

In [13]:
downloads = analysis.get_downloads()

marker = "SSU"

fasta_files = [download for download in downloads if isinstance(download, FastaDownload) and download.marker == marker]
mseq_files = [download for download in downloads if isinstance(download, MseqDownload) and download.marker == marker]
tsv_files = [download for download in downloads if isinstance(download, TsvDownload) and download.marker == marker]

Let's take a look at the number of rows in the fasta, mseq, and OTU files.

In [14]:
fasta = fasta_files[0].read_pandas()
fasta

,reference,sequence
0,SRR5788044.10095561-NS500496-106-HF3LCBGXX:2:1...,GCTCAGTAACACGTGGATTACCTGCCCTATCGTTCGGAATAACCTC...
1,SRR5788044.10101861-NS500496-106-HF3LCBGXX:2:1...,TCGATTAAGCCATGCGAGTCGAGAGTCCTCGGGACTCGGCATACTG...
2,SRR5788044.10217657-NS500496-106-HF3LCBGXX:2:1...,CTGAGACACGAATCCAGAACCTACGGGGTGCAGCAGGCGCGAAAAC...
3,SRR5788044.10249364-NS500496-106-HF3LCBGXX:2:1...,GCTGTAACTCGCCCTCGTGAAGCTGGATTCCGTAGTAATCGTGTTT...
4,SRR5788044.10249364-NS500496-106-HF3LCBGXX:2:1...,AGGCTGTAACTCGCCCTCGTGAAGCTGGATTCCGTAGTAATCGTGT...
...,...,...
27292,SRR5788044.9932850-NS500496-106-HF3LCBGXX:2:13...,AGTAAACTTTAAATCACTTTACGAGTATCAATTGGAGGGCAAGTCT...
27293,SRR5788044.9932850-NS500496-106-HF3LCBGXX:2:13...,ACCCTGGACTTTTACTTTGAGGAAATTAGTGTGTTTCAAGCAGGCT...
27294,SRR5788044.9948620-NS500496-106-HF3LCBGXX:2:13...,GACTAAGCCATGCATGTCTAAGTATAAGCAGATTATACTGCGAGAC...
27295,SRR5788044.9950945-NS500496-106-HF3LCBGXX:2:13...,AATCGAAGTCTTTAGGTGTCGACGGGGAGCTTGCTGCCAGGTGGAA...


In [15]:
fasta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27297 entries, 0 to 27296
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reference  27297 non-null  object
 1   sequence   27297 non-null  object
dtypes: object(2)
memory usage: 426.6+ KB


In [16]:
mseq = mseq_files[0].read()
mseq

,#query,dbhit,bitscore,identity,matches,mismatches,gaps,query_start,query_end,dbhit_start,dbhit_end,strand,Unnamed: 12,SILVA,Unnamed: 14
0,SRR5788044.10095561-NS500496-106-HF3LCBGXX:2:1...,JX281272.1.902,231,0.983264,235,4,0,0,239,82,321,+,NaN,sk__Archaea;k__;p__Euryarchaeota;c__;o__;f__;g...,NaN
1,SRR5788044.10101861-NS500496-106-HF3LCBGXX:2:1...,HQ529831.1.913,178,1.000000,178,0,0,0,178,37,215,+,NaN,sk__Archaea;k__;p__Euryarchaeota;c__;o__;f__;g...,NaN
2,SRR5788044.10217657-NS500496-106-HF3LCBGXX:2:1...,HQ529796.1.902,145,0.993197,146,1,0,0,147,285,432,+,NaN,sk__Archaea;k__;p__Euryarchaeota;c__;o__;f__;g...,NaN
3,SRR5788044.10249364-NS500496-106-HF3LCBGXX:2:1...,AACY020187844.576.1995,114,0.991379,115,1,0,0,116,1245,1361,+,NaN,sk__Archaea;k__;p__Euryarchaeota;c__;o__;f__;g...,NaN
4,SRR5788044.10249364-NS500496-106-HF3LCBGXX:2:1...,AACY020187844.576.1995,113,1.000000,113,0,0,1,114,1244,1357,+,NaN,sk__Archaea;k__;p__Euryarchaeota;c__;o__;f__;g...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27290,SRR5788044.9932850-NS500496-106-HF3LCBGXX:2:13...,FN669511.1.1796,131,1.000000,131,0,0,0,131,510,641,+,NaN,sk__Eukaryota;k__;p__;c__Dinophyceae,NaN
27291,SRR5788044.9932850-NS500496-106-HF3LCBGXX:2:13...,HQ438164.1.1618,126,0.977273,129,3,0,0,132,614,746,+,NaN,sk__Eukaryota;k__;p__;c__Dinophyceae,NaN
27292,SRR5788044.9948620-NS500496-106-HF3LCBGXX:2:13...,JX188316.1.1752,144,1.000000,144,0,0,0,144,21,165,+,NaN,sk__Eukaryota;k__;p__;c__;o__;f__;g__;s__,NaN
27293,SRR5788044.9950945-NS500496-106-HF3LCBGXX:2:13...,GU819708.1.1234,73,0.796748,98,25,0,0,123,511,634,+,NaN,sk__Eukaryota;k__,NaN


In [17]:
mseq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27295 entries, 0 to 27294
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   #query       27295 non-null  object 
 1   dbhit        27295 non-null  object 
 2   bitscore     27295 non-null  int64  
 3   identity     27295 non-null  float64
 4   matches      27295 non-null  int64  
 5   mismatches   27295 non-null  int64  
 6   gaps         27295 non-null  int64  
 7   query_start  27295 non-null  int64  
 8   query_end    27295 non-null  int64  
 9   dbhit_start  27295 non-null  int64  
 10  dbhit_end    27295 non-null  int64  
 11  strand       27295 non-null  object 
 12  Unnamed: 12  0 non-null      float64
 13  SILVA        27295 non-null  object 
 14  Unnamed: 14  0 non-null      float64
dtypes: float64(3), int64(8), object(4)
memory usage: 3.1+ MB


In [18]:
otu = tsv_files[0].read()
otu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   # OTU ID    328 non-null    int64  
 1   SRR5788044  328 non-null    float64
 2   taxonomy    328 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 7.8+ KB


So we go from 27,297 reads in fasta to 27,295 hits in the mseq file to just 328 OTUs.

The same number of distinct taxonomy values in the mseq file as well as the OTU suggests that the OTUs correspond to all distinct taxonomic assignments regardless of sequence similarity. All reads with assignment Bacteria for example are collapsed into a single OTU.

In [19]:
pd.Series([clean_taxonomy_string(tax) for tax in otu["taxonomy"]]).nunique()

328

In [20]:
pd.Series([clean_taxonomy_string(tax) for tax in mseq["SILVA"]]).nunique()

328

In [21]:
mseq_bacteria = mseq.loc[mseq["SILVA"].apply(lambda x: clean_taxonomy_string(x)) == "sk__Bacteria"]
mseq_bacteria

,#query,dbhit,bitscore,identity,matches,mismatches,gaps,query_start,query_end,dbhit_start,dbhit_end,strand,Unnamed: 12,SILVA,Unnamed: 14
751,SRR5788044.10002615-NS500496-106-HF3LCBGXX:2:1...,KX426402.1.1486,174,1.000000,174,0,0,0,174,1197,1371,+,NaN,sk__Bacteria;k__,NaN
761,SRR5788044.10009855-NS500496-106-HF3LCBGXX:2:1...,EU802338.1.1516,238,1.000000,238,0,0,0,238,1006,1244,+,NaN,sk__Bacteria;k__,NaN
774,SRR5788044.10023731-NS500496-106-HF3LCBGXX:2:1...,EU802327.1.1487,259,1.000000,259,0,0,0,259,1154,1413,+,NaN,sk__Bacteria;k__,NaN
777,SRR5788044.10028038-NS500496-106-HF3LCBGXX:2:1...,KC003366.1.1331,235,0.995781,236,1,0,0,237,166,403,+,NaN,sk__Bacteria;k__,NaN
800,SRR5788044.10057198-NS500496-106-HF3LCBGXX:2:1...,EU803059.1.1291,146,1.000000,146,0,0,0,146,927,1073,+,NaN,sk__Bacteria;k__,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24966,SRR5788044.9953649-NS500496-106-HF3LCBGXX:2:13...,JQ670970.1.1389,223,0.995556,224,1,0,0,225,839,1064,+,NaN,sk__Bacteria;k__,NaN
24992,SRR5788044.9976658-NS500496-106-HF3LCBGXX:2:13...,CP000097.1777273.1778730,252,0.988372,255,3,0,0,258,8,266,+,NaN,sk__Bacteria;k__,NaN
24997,SRR5788044.9984844-NS500496-106-HF3LCBGXX:2:13...,EU803059.1.1291,152,1.000000,152,0,0,0,152,892,1044,+,NaN,sk__Bacteria;k__,NaN
25004,SRR5788044.9997731-NS500496-106-HF3LCBGXX:2:13...,KU578488.1.1380,112,1.000000,112,0,0,0,112,1182,1294,+,NaN,sk__Bacteria;k__,NaN


In [22]:
otu_bacteria = otu.loc[otu["taxonomy"].apply(lambda x: clean_taxonomy_string(x)) == "sk__Bacteria"]
otu_bacteria

,# OTU ID,SRR5788044,taxonomy
6,103181,3269.0,sk__Bacteria


Grouping by taxonomy appears to be the approach currently used in [mgnify-to-dwc](https://github.com/gbif/mgnify-to-dwc), see occurrences for this sample here: <https://www.gbif.org/occurrence/search?dataset_key=f6da16a0-ad5a-4f47-a347-aa6281de3d0d&advanced=1&event_id=MGYA00593805>

## Alternative approach

Instead of relying on the OTU table as downloaded from MGnify, let's try grouping sequences by SILVA hit.

In [23]:
study = get_superstudy_studies("atlanteco")[0]
study


<Study https://www.ebi.ac.uk/metagenomics/studies/MGYS00005780 >

In [24]:
sample = study.get_samples(max_results=1)[0]
sample

<Sample https://www.ebi.ac.uk/metagenomics/samples/SRS2329696 >

In [25]:
run = sample.get_runs(max_results=1)[0]
run

<Run https://www.ebi.ac.uk/metagenomics/runs/SRR5788044 >

In [26]:
analysis = run.get_analyses(max_results=1)[0]
analysis

<Analysis https://www.ebi.ac.uk/metagenomics/analyses/MGYA00463299 >

In [27]:
marker = "LSU"
downloads = analysis.get_downloads()
fasta_files = [download for download in downloads if isinstance(download, FastaDownload) and download.marker == marker]
mseq_files = [download for download in downloads if isinstance(download, MseqDownload) and download.marker == marker]
fasta = fasta_files[0].read_pandas()
mseq = mseq_files[0].read()

Merge fasta and mseq tables:

In [28]:
df = fasta.merge(mseq.rename({"#query": "reference"}, axis=1), how="left", on="reference")

First clean taxonomy strings by removing empty ranks:

In [29]:
df["SILVA"] = [clean_taxonomy_string(tax) for tax in df["SILVA"]]

Let's take a look at the number of distinct sequences, taxonomy strings, and DB hits:

In [30]:
df["sequence"].nunique()

42037

In [31]:
df["SILVA"].nunique()

453

In [32]:
df["dbhit"].nunique()

3184

So an alternative approach would be to group sequences by DB hit, and pick a random representative sequence.